# Neuronale Netze - Einführung

In [ ]:
import torch
import numpy as np

from matplotlib import pyplot as plt

Neuronale Netze sind Funktionen: sie haben eine Ein- und Ausgabe.
Wir können sie z.B. 
- mit Bildern füttern, und sie sagen uns das Tier, das darauf abgebildet ist.
- mit Daten über eine Schüler\*in versorgen, und sie sagen uns die Abinote.
- nach einem Gedicht fragen, und sie schreiben uns ein Haiku.

Bevor sie dies machen können, müssen wir sie jedoch trainieren.
Unsere neuronalen Netzwerke bestehen aus mehreren Schichten von Neuronen, die jeweils mit allen Neuronen der benachbarten Schichten verbunden sind.
Wir überreichen die Eingabe an die erste Schicht (z.B. ein Pixel pro Neuron).
In jeder Schicht erhält nun das Neuron die Ausgaben der Vorgänger, gewichtet sie, 
addiert sie und fügt noch einen Bias hinzu.

Damit besteht ein Neuron aus einem Vektor $w$ an Gewichten und einer Zahl $b$ (Bias).
Gegeben eine Eingabe $x$ (die Ausgabe der Neuronen des vorherigen Layers) führt es nun die Berechnung
$w \cdot x + b$
aus.

In PyTorch können wir ein Neuron mit der Funktion `torch.nn.Linear` erzeugen:

In [ ]:
neuron = torch.nn.Linear(2,1)

Wir geben die Anzahl der Eingaben als erstes Argument an.
Das zweite Argument gibt die Anzahl der Neuronen an, die wir in der Schicht erzeugen wollen.

Die Gewichte und der Bias werden zufällig automatisch initialisiert, wir können sie aber auch überschreiben.
Wir setzen $w = (1, 2)$ und $b = 3$.

In [ ]:
w = torch.tensor([1.0,2.0])
b = torch.tensor(3.0)

neuron.weight.data = w
neuron.bias.data = b

Was ist die Ausgabe des Neurons für $x = (3,2)$? Versuch es erst im Kopf bevor du die Zelle ausführst. Vergiss den Bias nicht!

In [ ]:
x = torch.tensor([3.0, 2.0])

neuron(x).data

**Aufgabe**
- Erzeuge nun selbst einen Layer mit 10 Neuronen und je 3 Eingaben. 
- Gewichte und Bias brauchst du nicht anzupassen.
- Generiere eine Eingabe $x$ und schicke sie durch den Layer.

In [ ]:
layer = _ # Layer erzeugen
x = _     # passende Eingabe (Werte egal)
_         # x durch Layer schieben

Schau dir einmal die Gewichte und Bias an: wir haben separate Gewichte für jedes Neuron.

In [ ]:
layer.weight.data

In [ ]:
layer.bias.data

Du kannst natürlich auch mehrere Eingaben gleichzeitig verarbeiten. Um uns Arbeit zu sparen können wir zufällige Eingaben erzeugen. 

In [ ]:
x_rnd = torch.rand(3,5)
x_rnd

In [ ]:
layer(x_rnd)

Ups, das ging schief. Lies die Fehlermeldung und versuche den Fehler zu korrigieren.

## Aktivierungsfunktionen

Neuronen sollen die gesammelten Informationen erst ab einer gewissen Relevanz bzw. Stärke weitergeben.
Dies erreichen wir durch eine Aktivierungsfunktion. Nachdem das Neuron seine Eingaben aufsummiert hat, wird diese angewandt. Die am häufigsten verwendete ist ReLU: sie gibt nur positive Werte weiter:

In [ ]:
relu = torch.nn.ReLU()
three = torch.tensor([3])
print("relu(-3) =", relu(-three)[0])
print("relu( 3) =", relu(three)[0])

Versuche die ReLU-Funktion selbst zu implementieren:
1. `my_single_relu` für einzelne Zahlen
2. `my_tensor_relu` für torch Tensoren (nutze z.B. `torch.max`)

In [ ]:
def my_single_relu(x):
    return _ # <-- Lösung hier

In [ ]:
def my_tensor_relu(x):
    return _ # <-- Lösung hier

Teste die Funktionen auf verschiedenen Werten, es sollte immmer `True` ausgegeben werden:

In [ ]:
x_single = -32
x_tensor = torch.tensor([32, -32])

print(my_single_relu(x_single) == relu(torch.tensor(x_single)))
print(my_tensor_relu(x_tensor) == relu(x_tensor))

Was ist die Ableitung der ReLU-Funktion, können wir sie für die Eingabe $0$ bestimmen?
Argumentiere mit dem Differenzenquotienten.

## Netzwerke schichten

Wir können nun ganz einfach die Layer zu einem Netzwerk schichten: immer Neuronen und Aktivierungsfunktionen im Wechsel.

In [ ]:
net = torch.nn.Sequential(
    torch.nn.Linear(1, 16),  # <-- input layer
    torch.nn.ReLU(),# /      diese Zahlen müssen gleich sein
    torch.nn.Linear(16, 16), # <-- hidden layer
    torch.nn.ReLU(), # /     diese Zahlen müssen gleich sein
    torch.nn.Linear(16, 16), # <-- hidden layer
    torch.nn.ReLU(), # /     diese Zahlen müssen gleich sein
    torch.nn.Linear(16, 1)   # <-- output layer
)
net

Wir können nun eine Eingabe $x = 2$ durch das Netzwerk schicken, jedoch müssen wir sie dafür zuvor in einen Tensor packen:

In [ ]:
x = torch.tensor([2.0])
net(x)

Verdopple nun den vorletzten Layer auf eine Größe von 32:

In [ ]:
net_32 = torch.nn.Sequential(
    torch.nn.Linear(1, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 1)
)
net_32

In [ ]:
print("Korrekt!" if net_32[4].in_features == 32 else "Falsche Anzahl Neuronen")

x = torch.tensor([2.0])
net_32(x)

## Neuronale Netze Trainieren

Zu Beginn haben wir gesagt, dass neuronale Netze Funktionen sind, sie können sogar jede beliebige Funktion darstellen. Dafür müssen wir jedoch die Gewichte richtig setzen.
Die Gewichte richtig einzustellen ist von Hand jedoch schnell mühsam, bereits unser kleines Netz aus dem letzten Kapitel hat bereits zu viele Parameter (Gewichte inkl. Bias)! Wie viele eigentlich? (Tipp: Jedes Neuron hat ein Gewicht pro Eingabe, plus den Bias)

In [ ]:
num_weights = _ # <-- Lösung eintragen
num_weights == sum(p.numel() for p in net.parameters())

Wir brauchen also eine automatische Möglichkeit die Parameter zu optimieren (wir nennen das "trainieren").

Die Grundidee ist, dass wir Eingaben, für die wir das Ergebnis bereits kennen durch das Netzwerk schicken, und das Ergebnis des neuronalen Netzes mit dem erwarteten Wert vergleichen. Diese Daten heißen Trainingsdaten, ein Beispiel hier:

In [ ]:
data_train = torch.load('data/1d_dataset_train.pt')
x_train = data_train['x']
y_train = data_train['y']
plt.plot(x_train, y_train, '.', markersize=0.1)


Nun stellen wir die Gewichte ein (dazu später mehr), und prüfen mittels der Testdaten, wie nah wir schon an der Funktion sind. Zum Vergleich in blau die aktuellen (schlechten) Ergebnisse unseres Netzwerks:

In [ ]:
data_test = torch.load('data/1d_dataset_test.pt')
x_test = data_test['x']
y_test = data_test['y']

with torch.no_grad():
    y_pred_test = net(x_test)

plt.plot(x_test, y_pred_test, '.', markersize=0.1)
plt.plot(x_test, y_test, '.', markersize=0.1)

### Fehlerfunktion

Wir sehen z.B. für $x \approx 0.55$, dass wir ca. $0.82$ erwarten, unser Netzwerk jedoch ca. $0$ vorhersagt. Indem du `i` änderst, kannst andere Datenpunkte auswählen.

In [ ]:
i = 5
print("Eingabe: ", x_test[i].item())
print("Erwartete Ausgabe: ", y_test[i].item())
print("Vorhersage Netzwerk: ", y_pred_test[i].item())

Um auszudrücken, wie gut unsere aktuellen Vorhersagen sind, wollen wir den Fehler zwischen erwarteter Ausgabe und der Vorhersage berechnen. Dieser ist einfach der durchschnittliche Fehler gemittelt über alle Testdaten.

Für einen einzelnen Datenpunkt berechnen wir den Fehler als
$(Y^i_{\textit{test}} - Y^i_{\textit{pred}})^2$ (Squared Error).
Man könnte erwarten, dass der Fehler als $\|Y^i_{\textit{test}} - Y^i_{\textit{pred}}\|$ definiert sein sollte, aber damit lässt sich schlechter arbeiten, deswegen quadrieren wir.

Implementiere den Squared Error für einen Datenpunkt:

In [ ]:
def squared_err(x,y):
    return _ # <--- Lösung hier

y_test_i = 3.0
y_pred_i = 6.0

# Sollte 9.0 zurückgeben
squared_err(y_test_i, y_pred_i)

Programmiere nun den Mean Squared Error (MSE) als durchschnittlichen Fehler über alle Datenpunkte:

In [ ]:
def mse(x, y):
    # Lösung hier
    return _

# Sollte 5.0 zurückgeben
mse_x = torch.tensor([3.0, 4.0])
mse_y = torch.tensor([6.0, 5.0])

mse(mse_x, mse_y)

Natürlich ist der MSE in PyTorch bereits eingebaut:

In [ ]:
torch_mse = torch.nn.MSELoss()

torch_mse(mse_x, mse_y)

Wie du siehst, heißt die Funktion `MSELoss`. *Loss* bezeichnet in der Optimierung für den Fehler gegenüber der optimalen Lösung.

### Stochastic Gradient Descent

Wir wollen den Fehler optimieren, d.h. möglichst auf $0$ drücken. Dann sollte sich die blaue Kurve den roten Datenpunkten annähern.
Um das hinzubekommen können wir die Gewichte des neuronalen Netzwerks anpassen, die in jedem Neuron gespeichert sind.

Wir wählen zuerst ein paar zufällige Trainingsdaten (Minibatch) aus.

In [ ]:
batch_ind = torch.randint(len(x_train), (16,))
x_batch = x_train[batch_ind, :]
y_batch = y_train[batch_ind, :]
print(batch_ind)
print(x_batch)
print(y_batch)

Diese schicken wir durch das Netzwerk und bestimmen damit unsere Vorhersagen.

In [ ]:
y_pred = net(x_batch)
y_pred

Für jede Vorhersage wissen wir auch, was die korrekte Ausgabe wäre und können daher den Loss berechnen.
Diese Phase ist der Forward-Pass.

In [ ]:
loss = torch_mse(y_pred, y_batch)
loss

Nun bestimmen wir mittels Backpropagation, wie wir die Gewichte anpassen müssen, um den Loss ein klein wenig geringer zu machen.

In [ ]:
loss.backward()

Um zu bestimmen, wie genau die Gewichte angepasst werden sollen, benötigen wir einen Optimizer. Wir verwenden den sehr bekannten Algorithmus *Adam*. 

In [ ]:
optimizer = torch.optim.Adam(net.parameters())

print("Einige Parameter vor dem Schritt:")
print(list(net.parameters())[0].data)

optimizer.step()

print("\nEinige Parameter nach dem Schritt:")
print(list(net.parameters())[0].data)


Nach jedem Schritt müssen die Gradienten mittels `optimizer.zero_grad()` zurückgesetzt werden.

**Aufgabe** Implementiere den Trainingsalgorithmus und trainiere dein Netzwerk.

In [ ]:
from tqdm import tqdm

max_iter = 1000

for it in tqdm(range(max_iter)):
    # Erzeuge Mini-Batch
    
    # Vorhersage des Netzwerks
    
    # Loss berechnen

    # Backpropagation
    
    # Optimizer step


**Aufgabe** Berechne die Vorhersagen des Netzwerkes und bestimme den Loss auf dem Testset.

In [ ]:
with torch.no_grad():
    y_pred_test = _ # Lösung hier
    err = _ # Lösung hier
    print(err)

**Aufgabe** Erweitere deinen Trainingsalgorithmus, sodass schon während des Trainings die Performance des Netzes in regelmäßigen Abständen bestimmt wird und der Loss gespeichert wird, sodass du ihn nach dem Training plotten kannst.
Passe außerdem deine Netzwerkarchitektur und die Trainingsparameter an, sodass du bessere Ergebnisse bekommst.

Schritte:
1. Kopiere Algorithmus
2. Jeden 100ten Schritt: berechne Loss auf **Test**daten, gib ihn aus
3. Jeden 100ten Schritt: plotte Vorhersagen auf den **Test**daten vs. erwartete Ausgaben
4. In jedem Schritt: Speichere Loss auf **Trainings**daten in Liste.
5. Passe Netzwerkarchitektur + Parameter an

In [ ]:
# Loss während des Trainings plotten
plt.plot(_) # Trage hier dein Loss-Array ein
plt.show()

In [ ]:
# Loss auf Testdaten berechnen

# Ergebnisse grafisch darstellen

## Ziffern klassifizieren

### Aufgabe 7

Als nächstes widmen wir uns der Klassifikation von Ziffern. Wir verwenden hierfür den MNIST-Datensatz, den man sich über die Funktion [`torchvision.datasets.MNIST`](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST) herunterladen kann. Außerdem kannst du mit der Funktion direkt Transformationen auf dem Datensatz ausführen. Wir wollen die Bilder direkt mit [`torchvision.transforms.ToTensor`](https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor) zu Tensoren konvertieren.

In [ ]:
import torchvision

mnist_train = torchvision.datasets.MNIST('data/', train=True, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)
mnist_test = torchvision.datasets.MNIST('data/', train=False, 
                                         transform=torchvision.transforms.ToTensor(),
                                         download=True)

Einige Zifferen als Beispiele:

In [ ]:
for i in range(20):
    plt.imshow(torch.reshape(mnist_train[i][0], (28, 28)))
    plt.show()

    print(mnist_train[i][1])

Beim Training des letzten Modells haben wir die Mini-Batches manuell erzeugt. Allerdings gibt es die Funktion [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), die einem diese Aufgabe abnimmt.

**Aufgabe** Erzeuge einen solchen `DataLoader` für das Trainings- und Testset mit Batch-Größe 16. Achte darauf, dass (nur) die Trainingsdaten gemischt werden müssen. Schaue dafür ggf. in die Dokumentation/such im Internet/bei ChatGPT.

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=16)

Der Aufbau der neuronalen Netzes zur Klassifikation der Ziffern ist ähnlich dem im vorherigen Problem, allerdings müssen wir einige Dinge beachten.

Da es sich um Bilder handelt, müssen wir diese erst in Vektoren umwandeln. Das ist mit der Funktion [`torch.nn.Flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) möglich, die eine Schicht des neuronalen Netzwerks erzeugt.
Außerdem muss das Netzwerk jetzt nicht mehr eine sondern 10 Zahlen zurück geben.
Jede stellt die Wahrscheinlichkeit dar, dass die konkrete Eingabe die jeweilige Ziffer darstellt.

**Aufgabe** Implementiere ein solches Netz.

In [ ]:
net = _ # Lösung hier

net(torch.zeros(1, 28, 28)) # Test, sollte keinen Fehler liefern

**Aufgabe** Erstelle außerdem den Adam-`Optimizer` und den für Klassifikation benötigten [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [ ]:
loss_fn = _
optimizer = _

Um das Netzwerk zu trainieren, führen wir eine `for`-Schleife über den `DataLoader` aus. Damit iterieren wir einmal durch den gesamten Datensatz, was als eine Epoche bezeichnet wird. Mit einer zweiten `for`-Schleife können wir mehrere solcher Epochen ausführen.

**Aufgabe** Ergänze den Trainingsalgorithmus und trainiere dein Modell. Plotte anschließen den Loss.

In [ ]:
n_epoch = 5

loss_hist = []

for ep in range(n_epoch):
    for x_batch, y_batch in tqdm(train_loader):
        # Vorhersage des Netzwerks

        # Loss berechnen
        
        # Backpropagation
        

        # Optimizer step
        
        # Speichere Loss in loss_hist


In [ ]:
plt.plot(loss_hist)
plt.show()

**Aufgabe** Berechne die Genauigkeit deines Netzwerkes, also wie viele Bilder richtig klassifiziert werden, auf dem Testset.

In [ ]:
sum_correct = 0
sum_imgs = 0

for x_batch, y_batch in tqdm(test_loader):
    # Vorhersage des Netzes ohne Gradientenberechnung
    with torch.no_grad():
        y_pred = _
    
    # Vorhergesagtes Label
    y_pred = _
        
    # Anzahl der Bilder aktualisieren
    sum_imgs += _
    
    # Anzahl der korrekt klassifizierten Bilder
    sum_correct += _

# Accuracy berechnen und ausgeben
accuracy = _
print('Accuracy auf dem Testset: ', str(accuracy * 100) + "%")



**Aufgabe** Wiederhole das Training und berechne die Genauigkeit diesmal nach jeder Epoche. Verbessere außerdem deine Netzwerkarchitektur.

In [ ]:
# Du kannst deinen Code aus den vorherigen Aufgaben nutzen und anpassen.

# Bonus: Backpropagation selbst implementieren

Einige imports zu Beginn:

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Wir definieren eine einfache Funktion $f(x) := 3x^2 - 4x + 5$.

In [ ]:
def f(x):
  return 3*x**2 - 4*x + 5

print("f(3) =", f(3.0))


xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)
plt.show()

Bestimme die Ableitung bei $x = 3$ mit dem Differenzenquotienten:

In [ ]:
h = 0.0001
x = 3.0
dx = _ # Lösung hier
dx

Für zusammengesetzte Ausdrücke funktioniert das Ganze ebenfalls:

In [ ]:
# Eingaben
a = 2.0
b = -3.0
c = 10.0

d1 = a*b + c
c += h
d2 = a*b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1)/h)


Wir wollen nun automatisch die Gradienten für solche Ausdrücke bestimmen.
Dafür definieren wir eine Klasse `Value`, die diese repräsentiert:

In [ ]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._op = _op
    self.label = label
    self._prev = set(_children)
    self.grad = 0.0

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    return out
  
  def relu(self):
    x = self.data
    t = max(0,x)
    out = Value(t, (self, ), 'relu')
    
    return out

Wir können nun einen ähnlichen Ausdruck erzeugen und auswerten:

In [ ]:

a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10.0, label='c')
e = a*b; e.label = 'e'
d = e + c; d.label = 'd'
f = Value(-2.0, label='f')
L = d * f; L.label = 'L'
L

Um die Ausdrücke anschaulicher zu machen bauen wir eine Funktion, die diese zeichnet:

In [ ]:

import sys
!conda install --yes --prefix {sys.prefix} graphviz
!{sys.executable} -m pip install graphviz
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot


In [ ]:
draw_dot(L)

Versuche mit der `Value`-Klasse einen Fully Connected Layer mit 2 Inputs, einem Output und ReLU-Aktivierung zu definieren. Gibt den Knoten zufällige Werte und aussagekräftige Label.

In [ ]:
# inputs x1,x2
x1 = _
x2 = _
# weights w1,w2
w1 = _
w2 = _
# bias of the neuron
b = _
# x1*w1 + x2*w2 + b
x1w1 = _
x2w2 = _
x1w1x2w2 = _
n = _; n.label = 'n'
o = _; o.label = 'o'



Zeichne dein Netzwerk:

Wir berechnen die Ableitung jedes Knotens rückwärts, also von rechts nach links.
Angenommen, wir wissen bereits, wie sich die Ausgabe des Ausdrucks abhängig von $c = a + b$ ändert. Jetzt können wir auch bestimmen, wie sich die Ausgabe abhängig von $a$ oder $b$ ändert, ganz einfach mittels der Kettenregel:
Sei z.B. der Gradient von c $2$, wenn wir nun $c$ um $h = 0.0001$ erhöhen, erhöht sich auch die Ausgabe um ca. $2 * h$.
Wie erhöht sich die Ausgabe, wenn wir $a$ um $h$ erhöhen? Genau, auch um ca. $2 * h$, wir können also einfach den Gradienten von $c$ übernehmen.

Füge dies in der Klasse unten hinzu: 

In [ ]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._op = _op
    self.label = label
    self._prev = set(_children)
    self.grad = 0.0

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad = _ # übernimm out.grad
      other.grad = _ # übernimm out.grad
    out._backward = _backward
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    return out
  
  def relu(self):
    x = self.data
    t = max(0,x)
    out = Value(t, (self, ), 'relu')
    
    return out

Etwas komplizierter ist es bei der Multiplikation. Für $c = a * b$ und $a = -2$, Gradient von c = $3$ gilt, wenn wir $b$ um $h$ erhöhen, ändert sich die Ausgabe um $-2 * 3 * h = 6 * h$. Wir müssen also mit `a.data` multiplizieren.

Füge dies und die Ableitung von ReLU in der Klasse unten hinzu: 

In [ ]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._op = _op
    self.label = label
    self._backward = lambda: None
    self._prev = set(_children)
    self.grad = 0.0

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad = _ # übernimm out.grad
      other.grad = _ # übernimm out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += _
      other.grad += _
    out._backward = _backward
    
    return out
  
  def relu(self):
    x = self.data
    t = max(0,x)
    out = Value(t, (self, ), 'relu')
    
    def _backward():
      self.grad = _
    
    out._backward = _backward
    
    return out

Setze nun den Gradienten von $o = 0$ und ruf dann in der richtigen Reihenfolge (von rechts nach links) die Funktion `_backward` auf den Knoten auf. Überprüfe das Ergebnis auf Korrektheit.

Um nicht jedes Mal von Hand die `_backward`-Funktion aufrufen zu müssen, schreiben wir eine Funktion, die die richtige Reigenfolge berechnet und dann überall `_backward` aufruft.
Die Reihenfolge zu definieren geht so:

In [ ]:
topo = []
visited = set()
def build_topo(v):
  if v not in visited:
    _ # füge v zu visited hinzu
    for child in v._prev:
        _ # rekursiver Aufruf
    _ # füge v an topo an
build_topo(o)
list(reversed(topo))

Implementiere die Funktion `backward`, sodass sie die Topographie berechnet, dann den Gradienten des aktuellen Knoten auf $1.0$ setzt und schließlich überall `_backward()` aufruft

In [ ]:
class Value:
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self._op = _op
    self.label = label
    self._backward = lambda: None
    self._prev = set(_children)
    self.grad = 0.0

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad = 1.0 * out.grad # übernimm out.grad
      other.grad = 1.0 * out.grad # übernimm out.grad
    out._backward = _backward
    return out

  def __mul__(self, other):
    out = Value(self.data * other.data, (self, other), '*')
    return out
  
  def relu(self):
    x = self.data
    t = max(0,x)
    out = Value(t, (self, ), 'relu')
    
    return out

  def backward(self):
    _

Irgendetwas geht noch schief:

In [ ]:
a = Value(3.0, label='a')
b = a + a   ; b.label = 'b'
b.backward()
draw_dot(b)

Versuche den Fehler zu finden und zu korrigieren.
Tipp: `=` $\mapsto$ `+=`